In [1]:
import ee

# Trigger the authentication flow.
# ee.Authenticate()

ee.Initialize(project='ee-arzaaan789')

In [2]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("TGB_red_squirrel.csv", delimiter='\t')
df = df[df["occurrenceStatus"] == "PRESENT"]
df = df[df['year']>=2022]
# df['eventDate'] = df['eventDate'].str.replace('/','')
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')
df = df[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
df = df.dropna().reset_index(drop=True)

In [3]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

# ee.Initialize()

# Your dataframe 'df' must have columns: decimalLongitude, decimalLatitude
# Example: df = pd.read_csv("Erinaceus europaeus.csv", delimiter='\t')

def create_aoi(lon, lat, box_size_km=1):
    """Create approx 1km x 1km square polygon around lon, lat."""
    half_side_deg = box_size_km / 111.32 / 2  # Rough approx degrees per km

    coords = [
        [lon - half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat - half_side_deg]
    ]
    return ee.Geometry.Polygon(coords)

def compute_all_indices(feature, start_date, end_date):
    """Compute spectral indices and LST for one EE Feature (with AOI geometry)."""
    aoi = feature.geometry()

    s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .sort('CLOUDY_PIXEL_PERCENTAGE')
          .first())
    s2 = ee.Image(s2).clip(aoi)

    blue = s2.select('B2')
    green = s2.select('B3')
    red = s2.select('B4')
    nir = s2.select('B8')
    swir = s2.select('B11')

    L = 0.5  # SAVI constant

    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    savi = nir.subtract(red).divide(nir.add(red).add(L)).multiply(1 + L).rename('SAVI')
    mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    bsi = (red.add(blue).subtract(nir.add(swir))).divide(red.add(blue).add(nir).add(swir)).rename('BSI')
    ui = nir.subtract(swir).divide(nir.add(swir)).rename('UI')

    reducers = ee.Reducer.mean()
    scale_10m = 1000

    ndvi_mean = ndvi.reduceRegion(reducers, aoi, scale_10m).get('NDVI')
    ndwi_mean = ndwi.reduceRegion(reducers, aoi, scale_10m).get('NDWI')
    ndbi_mean = ndbi.reduceRegion(reducers, aoi, scale_10m).get('NDBI')
    savi_mean = savi.reduceRegion(reducers, aoi, scale_10m).get('SAVI')
    mndwi_mean = mndwi.reduceRegion(reducers, aoi, scale_10m).get('MNDWI')
    ndsi_mean = ndsi.reduceRegion(reducers, aoi, scale_10m).get('NDSI')
    bsi_mean = bsi.reduceRegion(reducers, aoi, scale_10m).get('BSI')
    ui_mean = ui.reduceRegion(reducers, aoi, scale_10m).get('UI')

    # MODIS LST dataset
    modis = (ee.ImageCollection("MODIS/061/MOD11A1")
             .filterBounds(aoi)
             .filterDate(start_date, end_date)
             .select('LST_Day_1km'))

    lst_mean_img = modis.mean().multiply(0.02).clip(aoi)

    lst_mean = lst_mean_img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000
    ).get('LST_Day_1km')

    return feature.set({
        'NDVI': ndvi_mean,
        'NDWI': ndwi_mean,
        'NDBI': ndbi_mean,
        'SAVI': savi_mean,
        'MNDWI': mndwi_mean,
        'NDSI': ndsi_mean,
        'BSI': bsi_mean,
        'UI': ui_mean,
        'LST': lst_mean
    })

# Split df into batches
batch_size = 50
batches = [df.iloc[i:i+batch_size] for i in range(0, len(df), batch_size)]

results_list = []

index_names = ['NDVI', 'NDWI', 'NDBI', 'SAVI', 'MNDWI', 'NDSI', 'BSI', 'UI', 'LST']

for batch in tqdm(batches, desc="Processing batches"):
    features = []
    batch_indices = []
    feature_metadata = {}

    for idx, row in batch.iterrows():
        
        aoi = create_aoi(row['decimalLongitude'], row['decimalLatitude'])
        feature = ee.Feature(aoi).set('index', idx)
        features.append(feature)
        batch_indices.append(idx)  # Save the original index
        
        event_date = row['eventDate']
        start_date = (event_date - timedelta(days=15)).strftime('%Y-%m-%d')
        end_date = (event_date + timedelta(days=15)).strftime('%Y-%m-%d')
        feature_metadata[idx] = (start_date, end_date)

    # Create a FeatureCollection from the list of features
    fc = ee.FeatureCollection(features)
    
    # Define wrapper for map to inject per-feature dates
    def map_with_dates(f):
        idx = f.get('index')
        # Use dictionary lookup to get dates for this feature
        date_dict = ee.Dictionary(ee.Dictionary(feature_metadata))
        dates = ee.List(date_dict.get(ee.Number(idx)))
        return compute_all_indices(f, dates.get(0), dates.get(1))

    try:
        result_fc = fc.map(map_with_dates)
        results = result_fc.getInfo()

        rows = []
        for f in results['features']:
            props = f['properties']
            rows.append(props)

    except Exception as e:
        print(f"Error processing batch starting at index {batch.index[0]}: {e}")
        # If there's an error, create placeholder rows with None
        rows = [{'index': i, **{name: None for name in index_names}} for i in batch_indices]

    batch_results_df = pd.DataFrame(rows).sort_values('index').reset_index(drop=True)
    results_list.append(batch_results_df)

# Concatenate all batches and sort by original index
all_results_df = pd.concat(results_list).sort_values('index').reset_index(drop=True)

# Merge with original df
df_final = pd.concat([df.reset_index(drop=True), all_results_df.drop(columns=['index'])], axis=1)

print(df_final.head())


Processing batches: 100%|██████████| 237/237 [16:39<00:00,  4.22s/it]

                    species  decimalLatitude  decimalLongitude  eventDate  \
0                 Glis glis            47.60             14.72 2022-09-16   
1  Muscardinus avellanarius            47.59             14.63 2022-08-04   
2  Muscardinus avellanarius            47.59             14.63 2022-08-04   
3  Muscardinus avellanarius            47.61             14.73 2022-08-09   
4  Muscardinus avellanarius            50.93             -4.04 2025-04-25   

        BSI         LST     MNDWI      NDBI      NDSI      NDVI      NDWI  \
0  0.051256  288.906226  0.150759 -0.192576  0.150759  0.060848 -0.042321   
1 -0.736161  296.289533 -0.560711 -0.213792 -0.560711  0.724840 -0.688657   
2 -0.736161  296.289533 -0.560711 -0.213792 -0.560711  0.724840 -0.688657   
3 -0.847292  296.983011 -0.675923 -0.249017 -0.675923  0.848991 -0.791385   
4 -0.726693  290.634316 -0.570324 -0.224353 -0.570324  0.737604 -0.704527   

       SAVI        UI  
0  0.091262  0.192576  
1  1.087059  0.213792  
2 

In [4]:
df_final.to_csv("tgb_red_squirrel_full_data.csv", index=False)
len(df_final)
df = pd.read_csv("tgb_red_squirrel_full_data.csv")
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')

df = df.dropna().reset_index(drop=True)

In [5]:
sitka_spruce = pd.read_csv("Sitka spruce.csv", delimiter='\t')
sitka_spruce = sitka_spruce[sitka_spruce["occurrenceStatus"] == "PRESENT"]
sitka_spruce = sitka_spruce[sitka_spruce['year']>=2022]
sitka_spruce['eventDate'] = pd.to_datetime(sitka_spruce['eventDate'], format='%Y-%m-%d', errors='coerce')
sitka_spruce = sitka_spruce[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
sitka_spruce = sitka_spruce.dropna().reset_index(drop=True)

grey_squirrel = pd.read_csv("grey squirrel.csv", delimiter='\t')
grey_squirrel = grey_squirrel[grey_squirrel["occurrenceStatus"] == "PRESENT"]
grey_squirrel = grey_squirrel[grey_squirrel['year']>=2022]
grey_squirrel['eventDate'] = pd.to_datetime(grey_squirrel['eventDate'], format='%Y-%m-%d', errors='coerce')
grey_squirrel = grey_squirrel[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
grey_squirrel = grey_squirrel.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_84777/983191898.py:1: DtypeWarning: Columns (41,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  sitka_spruce = pd.read_csv("Sitka spruce.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_84777/983191898.py:8: DtypeWarning: Columns (2,10,36,37,38,39,40,41,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  grey_squirrel = pd.read_csv("grey squirrel.csv", delimiter='\t')


In [6]:
import numpy as np
from sklearn.neighbors import BallTree

# Convert lat/lon to radians for BallTree
red_squirrel_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
sitka_spruce_coords = np.deg2rad(sitka_spruce[['decimalLatitude', 'decimalLongitude']].values)
grey_squirrel_coords = np.deg2rad(grey_squirrel[['decimalLatitude', 'decimalLongitude']].values)


# Build BallTree using haversine metric
tree_sitka_spruce = BallTree(sitka_spruce_coords, metric='haversine')
tree_grey_squirrel = BallTree(grey_squirrel_coords, metric='haversine')

# Define search radius: 1 km in radians
earth_radius_km = 6371.0
radius = 1 / earth_radius_km  # 1 km in radians

# Initialize presence columns
df['sitka_spruce_presence'] = 0
df['grey_squirrel_presence'] = 0

# Iterate through each red squirrel point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = red_squirrel_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # SITKA SPRUCE
    idxs = tree_sitka_spruce.query_radius(point, r=radius)[0]
    for j in idxs:
        sitka_date = sitka_spruce.loc[j, 'eventDate']
        if sitka_date.month == event_month and sitka_date.year == event_year:
            df.at[i, 'sitka_spruce_presence'] = 1
            break  # Found at least one match, no need to check further

    # GREY SQUIRREL
    idxs = tree_grey_squirrel.query_radius(point, r=radius)[0]
    for j in idxs:
        grey_squirrel_date = grey_squirrel.loc[j, 'eventDate']
        if grey_squirrel_date.month == event_month and grey_squirrel_date.year == event_year:
            df.at[i, 'grey_squirrel_presence'] = 1
            break

100%|██████████| 11500/11500 [00:01<00:00, 6763.20it/s]


In [7]:
from pyproj import Transformer
from rasterio.windows import Window
import rasterio
from collections import Counter
import numpy as np


land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(df['decimalLongitude'], df['decimalLatitude']))
df['easting_ni'], df['northing_ni'] = zip(*transformer_ni.itransform(coords))
df['easting_gb'], df['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class_1km(row):
    try:
        window_size_pixels = 100  # 1 km / 10m resolution

        # Open GB raster and read 1km x 1km window
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])

            # Calculate window bounds, making sure not to go out of raster bounds
            row_start = max(row_idx - window_size_pixels // 2, 0)
            col_start = max(col_idx - window_size_pixels // 2, 0)

            # Adjust window size if near edges
            height = min(window_size_pixels, src.height - row_start)
            width = min(window_size_pixels, src.width - col_start)

            window = Window(col_start, row_start, width, height)
            data = src.read(1, window=window)

            # If all zero (or no data), fallback to NI raster
            if np.all(data == 0):
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx_ni, col_idx_ni = src_ni.index(row['easting_ni'], row['northing_ni'])

                    row_start_ni = max(row_idx_ni - window_size_pixels // 2, 0)
                    col_start_ni = max(col_idx_ni - window_size_pixels // 2, 0)

                    height_ni = min(window_size_pixels, src_ni.height - row_start_ni)
                    width_ni = min(window_size_pixels, src_ni.width - col_start_ni)

                    window_ni = Window(col_start_ni, row_start_ni, width_ni, height_ni)
                    data_ni = src_ni.read(1, window=window_ni)

                    data = data_ni

            # Find the most common class (mode) ignoring zeros (assuming 0 means no data)
            unique, counts = np.unique(data[data != 0], return_counts=True)
            if len(counts) == 0:
                return "Unknown"

            mode_class = unique[np.argmax(counts)]
            return land_cover_map.get(mode_class, "Unknown")

    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


tqdm.pandas()
df['Land_cover'] = df.progress_apply(get_land_cover_class_1km, axis=1)


  0%|          | 11/11500 [00:00<04:06, 46.61it/s]

Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative


  1%|          | 63/11500 [00:00<02:01, 94.41it/s]

Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of co

  1%|          | 139/11500 [00:01<00:40, 279.53it/s]

...

Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of co

 99%|█████████▉| 11362/11500 [02:39<00:00, 510.28it/s]

Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of co

100%|██████████| 11500/11500 [02:40<00:00, 71.61it/s] 

Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of columns or rows must be non-negative
Error processing row: Number of co

In [8]:
df = df.dropna()
df=df.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
df = df[df['Land_cover'] != "Unknown"].reset_index(drop=True)
df.to_csv("tgb_red_squirrel_final_data.csv", index=False)